In [6]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [10]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
# cfg = Config.fromfile('./configs/_practice/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/_practice/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco.py')

root='../../dataset/'

epoch = 'latest'

# # dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco'

# cfg.model.roi_head.bbox_head.num_classes = 10

# cfg.optimizer_config.grad_clip = dict(max_norm=10, norm_type=2)
# cfg.model.train_cfg = None

In [11]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [12]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/detection/hongrok/mmdetection/mmdet/models/dense_heads/anchor_head.py:94: UserWarning: DeprecationWarning: Determining whether to samplingby loss type is deprecated, please delete sampler inyour config when using `FocalLoss`, `GHMC`, `QualityFocalLoss` or other FocalLoss variant.
  'DeprecationWarning: Determining whether to sampling'


load checkpoint from local path: ./work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/latest.pth


In [24]:
import json
out_name = './work_dirs/outputs/output'
with open(f'{out_name}.bbox.json', mode='r') as f:
            output2 = json.load(f)

In [25]:
pd.DataFrame(output2)

,image_id,bbox,score,category_id
0,0,"[22.134185791015625, 279.5380859375, 226.82571...",0.104458,0
1,0,"[240.3704833984375, 677.2055053710938, 62.3079...",0.082760,0
2,0,"[517.0897216796875, 112.08084106445312, 265.70...",0.628697,1
3,0,"[577.43603515625, 239.44430541992188, 178.4556...",0.508581,1
4,0,"[532.5201416015625, 128.9481201171875, 228.412...",0.395894,1
...,...,...,...,...
110950,4870,"[337.1630859375, 194.45150756835938, 311.15222...",0.111344,3
110951,4870,"[337.1630859375, 194.45150756835938, 311.15222...",0.095029,4
110952,4870,"[561.674072265625, 534.924072265625, 64.326171...",0.078359,4
110953,4870,"[331.6002197265625, 197.4058837890625, 322.031...",0.217344,5


In [15]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 17.7 task/s, elapsed: 275s, ETA:     0s

In [42]:
import pickle

with open('./work_dirs/outputs/output.pkl', mode='rb') as f:
    output2 = pickle.load(f)

In [43]:
output2

[[array([[2.21341858e+01, 2.79538086e+02, 2.48959900e+02, 4.40542053e+02,
          1.04457796e-01],
         [2.40370483e+02, 6.77205505e+02, 3.02678467e+02, 7.43421692e+02,
          8.27603936e-02]], dtype=float32),
  array([[5.17089722e+02, 1.12080841e+02, 7.82798218e+02, 3.49588928e+02,
          6.28697455e-01],
         [5.77436035e+02, 2.39444305e+02, 7.55891724e+02, 3.47555450e+02,
          5.08581221e-01],
         [5.32520142e+02, 1.28948120e+02, 7.60933105e+02, 2.45438751e+02,
          3.95894378e-01],
         [6.23016602e+02, 2.54168243e+02, 7.31081665e+02, 3.40172333e+02,
          2.94419855e-01],
         [1.94703674e+01, 2.90255554e+02, 2.53297913e+02, 4.40333313e+02,
          2.53075331e-01],
         [6.50540588e+02, 1.76950531e+02, 7.28421448e+02, 2.56656952e+02,
          2.38162085e-01],
         [3.94019714e+02, 4.84848907e+02, 6.06240479e+02, 5.72455811e+02,
          1.53751567e-01],
         [5.55048279e+02, 1.44349655e+02, 6.09875549e+02, 2.58200745e+02,


In [23]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
# submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.104457796 22.134186 279.5381 248.9599 440....,test/0000.jpg
1,0 0.16608281 324.83737 257.4563 764.897 705.42...,test/0001.jpg
2,0 0.23000312 408.55847 268.46582 662.2739 329....,test/0002.jpg
3,1 0.23092659 0.0 699.2534 303.1502 1024.0 1 0....,test/0003.jpg
4,0 0.094343066 180.84293 314.67935 921.0951 753...,test/0004.jpg


In [38]:
output[0][9].shape

(0, 5)

In [26]:
submission

,PredictionString,image_id
0,0 0.104457796 22.134186 279.5381 248.9599 440....,test/0000.jpg
1,0 0.16608281 324.83737 257.4563 764.897 705.42...,test/0001.jpg
2,0 0.23000312 408.55847 268.46582 662.2739 329....,test/0002.jpg
3,1 0.23092659 0.0 699.2534 303.1502 1024.0 1 0....,test/0003.jpg
4,0 0.094343066 180.84293 314.67935 921.0951 753...,test/0004.jpg
...,...,...
4866,0 0.32705113 246.18771 384.3518 714.7771 692.2...,test/4866.jpg
4867,0 0.11603857 347.7578 341.5201 804.68506 702.6...,test/4867.jpg
4868,0 0.1139731 449.27673 465.48517 792.9005 834.8...,test/4868.jpg
4869,0 0.10588968 0.0 207.58191 1024.0 933.84937 0 ...,test/4869.jpg


In [32]:
pd.DataFrame(output2)

,image_id,bbox,score,category_id
0,0,"[22.134185791015625, 279.5380859375, 226.82571...",0.104458,0
1,0,"[240.3704833984375, 677.2055053710938, 62.3079...",0.082760,0
2,0,"[517.0897216796875, 112.08084106445312, 265.70...",0.628697,1
3,0,"[577.43603515625, 239.44430541992188, 178.4556...",0.508581,1
4,0,"[532.5201416015625, 128.9481201171875, 228.412...",0.395894,1
...,...,...,...,...
110950,4870,"[337.1630859375, 194.45150756835938, 311.15222...",0.111344,3
110951,4870,"[337.1630859375, 194.45150756835938, 311.15222...",0.095029,4
110952,4870,"[561.674072265625, 534.924072265625, 64.326171...",0.078359,4
110953,4870,"[331.6002197265625, 197.4058837890625, 322.031...",0.217344,5


In [44]:
config_file='configs/_practice/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco.py' # config file
aaa = Config.fromfile(config_file) 

In [49]:
aaa.keys()

dict_keys(['model', 'dataset_type', 'data_root', 'classes', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'opencv_num_threads', 'mp_start_method', 'pretrained'])